In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")

In [0]:
data = data.fillna(method="ffill")

In [0]:
words = list(set(data["Word"].values))
n_words = len(words); n_words

35178

In [0]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

17

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:

getter = SentenceGetter(data)

In [0]:
sent = getter.get_next()

In [0]:
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [0]:
sentences = getter.sentences

In [0]:
sentences[:10]

[[('Thousands', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('demonstrators', 'NNS', 'O'),
  ('have', 'VBP', 'O'),
  ('marched', 'VBN', 'O'),
  ('through', 'IN', 'O'),
  ('London', 'NNP', 'B-geo'),
  ('to', 'TO', 'O'),
  ('protest', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('war', 'NN', 'O'),
  ('in', 'IN', 'O'),
  ('Iraq', 'NNP', 'B-geo'),
  ('and', 'CC', 'O'),
  ('demand', 'VB', 'O'),
  ('the', 'DT', 'O'),
  ('withdrawal', 'NN', 'O'),
  ('of', 'IN', 'O'),
  ('British', 'JJ', 'B-gpe'),
  ('troops', 'NNS', 'O'),
  ('from', 'IN', 'O'),
  ('that', 'DT', 'O'),
  ('country', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Iranian', 'JJ', 'B-gpe'),
  ('officials', 'NNS', 'O'),
  ('say', 'VBP', 'O'),
  ('they', 'PRP', 'O'),
  ('expect', 'VBP', 'O'),
  ('to', 'TO', 'O'),
  ('get', 'VB', 'O'),
  ('access', 'NN', 'O'),
  ('to', 'TO', 'O'),
  ('sealed', 'JJ', 'O'),
  ('sensitive', 'JJ', 'O'),
  ('parts', 'NNS', 'O'),
  ('of', 'IN', 'O'),
  ('the', 'DT', 'O'),
  ('plant', 'NN', 'O'),
  ('Wednesday', 'NNP', 'B-tim'),
  ('

In [0]:
max_len = 75
max_len_char = 10

In [0]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [0]:
print(word2idx["Obama"])
print(tag2idx["B-geo"])

4936
11


In [0]:
from keras.preprocessing.sequence import pad_sequences
X_word = [[word2idx[w[0]] for w in s] for s in sentences]

In [0]:
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["PAD"], padding='post', truncating='post')

In [0]:
chars = set([w_i for w in words for w_i in w])
n_chars = len(chars)
print(n_chars)

98


In [0]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}

char2idx["UNK"] = 1
char2idx["PAD"] = 0
  

In [0]:
X_char = []
for sentence in sentences:
    sent_seq = []
    for i in range(max_len):#max_len=75
        word_seq = []
        for j in range(max_len_char):#max_len_char=10
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [0]:
char2idx.get(sentences[2][0][1])

In [0]:

sentences[2][0][1]

'NN'

In [0]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [0]:
x=[]
for s in sentences:
  for w in s:
    x.append(tag2idx[w[2]])

In [0]:
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')

In [0]:
from sklearn.model_selection import train_test_split

In [0]:

X_word_tr, X_word_te, y_tr, y_te = train_test_split(X_word, y, test_size=0.1, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.1, random_state=2018)

In [0]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

In [0]:
# input and embedding for words
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

#input dim=35178, output dim= 20 , param= ip*op

In [0]:
# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=10,
                           input_length=max_len_char, mask_zero=True))(char_in)
#ip=  98+2,op=10,param=ip*op

In [0]:
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

#ip= 10,op=20 ,param= 4*(ip*op +(op)^2)

In [0]:
# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
#ip=20 op=50 , param= 8*(ip*op +(op)^2)

In [0]:
#param=1818, ip=100,op=18,param= 100*18 +18

out = TimeDistributed(Dense(n_tags + 1, activation="softmax"))(main_lstm)

model = Model([word_in, char_in], out)

In [0]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")

In [111]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 75, 10)       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 75)           0                                            
__________________________________________________________________________________________________
time_distributed_13 (TimeDistri (None, 75, 10, 10)   1000        input_9[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 75, 20)       703600      input_8[0][0]                    
__________________________________________________________________________________________________
time_distr

In [96]:
history = model.fit([X_word_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    np.array(y_tr).reshape(len(y_tr), max_len, 1),
                    batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Train on 38846 samples, validate on 4317 samples
Epoch 1/10
38846/38846 [==============================] - 285s 7ms/step - loss: 0.4336 - val_loss: 0.1847
Epoch 2/10
38846/38846 [==============================] - 274s 7ms/step - loss: 0.1540 - val_loss: 0.1291
Epoch 3/10
38846/38846 [==============================] - 273s 7ms/step - loss: 0.1156 - val_loss: 0.1118
Epoch 4/10
38846/38846 [==============================] - 271s 7ms/step - loss: 0.0986 - val_loss: 0.1058
Epoch 5/10
38846/38846 [==============================] - 276s 7ms/step - loss: 0.0887 - val_loss: 0.1027
Epoch 6/10
38846/38846 [==============================] - 274s 7ms/step - loss: 0.0820 - val_loss: 0.1010
Epoch 7/10
38846/38846 [==============================] - 265s 7ms/step - loss: 0.0769 - val_loss: 0.1000
Epoch 8/10
38846/38846 [==============================] - 266s 7ms/step - loss: 0.0727 - val_loss: 0.0985
Epoch 9/10
38846/38846 [==============================] - 264s 7ms/step - loss: 0.0693 - val_loss: 0.09

In [0]:
hist = pd.DataFrame(history.history)

In [0]:
model.save('new.h5')

In [115]:
i = 1923
p = np.argmax(y_pred[i], axis=-1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_word_te[i], y_te[i], p):
    if w != 0:
        print("{:15}: {:5} {}".format(idx2word[w], idx2tag[t], idx2tag[pred]))

Word           ||True ||Pred
North          : B-geo B-geo
Korea          : I-geo I-geo
's             : O     O
nuclear        : O     O
weapons        : O     O
program        : O     O
will           : O     O
be             : O     O
the            : O     O
focus          : O     O
of             : O     O
her            : O     O
meetings       : O     O
in             : O     O
Tokyo          : B-geo B-geo
,              : O     O
Seoul          : B-geo B-geo
and            : O     O
Beijing        : B-geo B-geo
.              : O     O


In [116]:
!git remote add origin https://github.com/mansi1189/Natural-Language-Processing.git

fatal: not a git repository (or any of the parent directories): .git


In [119]:
!echo "# Natural-Language-Processing" >> README.md
!git init
# !git add README.md
!git commit -m "first commit"
!git remote add origin https://github.com/mansi1189/Natural-Language-Processing.git
!git push -u origin master

Reinitialized existing Git repository in /content/.git/
[master (root-commit) 2899f12] first commit
 1 file changed, 1 insertion(+)
 create mode 100644 README.md
fatal: remote origin already exists.
fatal: could not read Username for 'https://github.com': No such device or address


In [0]:
!git config --global user.email "mansi.deolalikar@gmail.com"
!git config --global user.name "Mansi Deolalikar"

In [0]:
""